<a href="https://colab.research.google.com/github/hope04302/freeSearch/blob/main/commonAI/nsmcTest1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 실험 1 (5.24.)

- 본격적인 실험에 앞서, 데이터셋 분석

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math
import random as rd

In [ ]:
path_to_train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
path_to_test_file = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

train_text = open(path_to_train_file, 'rb').read().decode(encoding='utf-8')
test_text = open(path_to_test_file, 'rb').read().decode(encoding='utf-8')

train_text_x = [row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t') > 0]
train_y = np.array([[0, 1] if int(row.split('\t')[2]) else [1, 0] for row in train_text.split('\n')[1:] if row.count('\t') > 0])

test_text_x = [row.split('\t')[1] for row in test_text.split('\n')[1:] if row.count('\t') > 0]
test_y = np.array([[0, 1] if int(row.split('\t')[2]) else [1, 0] for row in test_text.split('\n')[1:] if row.count('\t') > 0])

for i in range(10):
    print(train_text_x[i], train_y[i])

4893335/4893335 [==============================] - 0s 0us/step
아 더빙.. 진짜 짜증나네요 목소리 [1 0]
흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나 [0 1]
너무재밓었다그래서보는것을추천한다 [1 0]
교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정 [1 0]
사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다 [0 1]
막 걸음마 뗀 3세부터 초등학교 1학년생인 8살용영화.ㅋㅋㅋ...별반개도 아까움. [1 0]
원작의 긴장감을 제대로 살려내지못했다. [1 0]
별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지..정말 발로해도 그것보단 낫겟다 납치.감금만반복반복..이드라마는 가족도없다 연기못하는사람만모엿네 [1 0]
액션이 없는데도 재미 있는 몇안되는 영화 [0 1]
왜케 평점이 낮은건데? 꽤 볼만한데.. 헐리우드식 화려함에만 너무 길들여져 있나? [0 1]


악플, 선플 수 비교

In [ ]:
print(np.sum(train_y, axis=0), np.sum(test_y, axis=0))

[75173 74827] [24827 25173]


쓰여있는 단어 분석

In [ ]:
words = set()
for i in train_text_x:
    words |= set(i)
print(len(words))
print(words)

3004
{'럤', '헐', '봇', '셸', '쀼', '찿', '맠', '똭', '훕', '광', '갘', '든', '쥣', '짤', '볻', '쫌', '쏴', '놨', 'そ', '썱', '됑', '촤', '즤', '콴', '껬', '샹', '갰', '퓌', '定', '빳', '홱', '앟', '셑', 'G', '밭', '↓', '맸', '갉', '곻', '좊', '뮈', '치', '맜', '횔', '췃', '존', '둔', '롯', '잼', '댑', '팡', '꺤', '벌', '뵙', '떈', '일', '렬', '끗', '싑', '려', '숭', '쒜', 'g', '읍', '쥴', '펑', '룽', '톱', '꿇', '멸', '알', ' ', '윈', '最', '꿍', '맞', '녿', '깞', '老', '8', 'N', '윽', '앰', '祖', '부', '톡', '늡', 'ㅄ', '범', '씻', '信', '핓', '삔', '훸', '갖', '퐛', '힉', '봄', '→', '응', 'ㅕ', '땋', '쳫', '밫', '훤', '佑', '팔', 'ㅠ', '숍', '욈', '웜', '뵛', '징', '켯', '안', '숫', '햐', '빙', 'た', '밬', '햏', '핧', '값', '쨈', '됫', 'も', 'リ', '뗴', '넒', 'の', '팜', '뚝', '륨', '닁', '체', '숙', '맘', '윖', '後', '런', '뿐', '탆', '첩', '퉤', '불', '련', '꼭', '峽', '혔', '꽉', '깼', '굉', '핦', '팩', '져', '픕', '+', '憎', '뭨', '럈', '꼅', '옳', '섀', '÷', '씝', '월', "'", '믐', '쭘', 'ゃ', 'ㅂ', '좡', '색', '뱀', '싷', '푸', '붹', '엑', '증', '@', '헠', '걔', '웃', '빤', '딴', '믄', '율', '쿈', '곽', '궅', '줫', '춥', '곱', '젆', '낔', '쑴', '뵤', '失', '찯',

## 실험 2

5월 24일

- 음절 단위로 토큰화, 2000 이내 글자 제외하고는 `<unk>`로 처리.
- 시작부분이 `<st>`, 문장 끝마다 `<ed>` 추가, 띄어쓰기 무시
(처음에는 kss를 쓸려고 했는데 너무 느려서 바꿈)
- 140자를 기준으로 뒤에 패딩(왜냐하면, 최대 길이가 140자에 근접)
- 모델은 아래와 같음

```
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(2500, 300, input_length=140),
    tf.keras.layers.LSTM(units=50),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=50, batch_size=64, validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

```


In [ ]:
%%bash
pip install kss

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.3/131.3 kB 11.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.4/26.4 MB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kss: filename=kss-4.5.3-py3-none-any.whl size=54258 sha256=9ab765fb350530d718d7c7750814ccfb781e40e922f8ca66cf7d372c045cb5eb
  Stored in directory: /root/.cache/pip/wheels/d8/9e/a3/5b09e3f14722fa0d77f47fe840668d426760023bdd11b0fbd9
  Created wheel for pecab: filename=pecab-1.0.8-py3-none-any.whl size=26646666 sha256=987544e3dc0db0a6b5c42882379936bb9db5d3d47463635b4d85dd5b1581e858
  Stored in directory: /root/.cache/pip/wheels/5c/6f/b

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import math
import random as rd
from kss import split_sentences

In [ ]:
%%bash
pip install kiwipiepy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 30.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602629 sha256=da8466c6fbaad35cb7c773f1cef46336979764445dacf35b2a02ada2b4583175
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


### 주요 함수

In [ ]:
path_to_train_file = tf.keras.utils.get_file('train.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt')
path_to_test_file = tf.keras.utils.get_file('test.txt', 'https://raw.githubusercontent.com/e9t/nsmc/master/ratings_test.txt')

train_text = open(path_to_train_file, 'rb').read().decode(encoding='utf-8')
test_text = open(path_to_test_file, 'rb').read().decode(encoding='utf-8')

train_text_x = [row.split('\t')[1] for row in train_text.split('\n')[1:] if row.count('\t') > 0]
train_y = np.array([[0, 1] if int(row.split('\t')[2]) else [1, 0] for row in train_text.split('\n')[1:] if row.count('\t') > 0])

test_text_x = [row.split('\t')[1] for row in test_text.split('\n')[1:] if row.count('\t') > 0]
test_y = np.array([[0, 1] if int(row.split('\t')[2]) else [1, 0] for row in test_text.split('\n')[1:] if row.count('\t') > 0])

print(len(train_y), len(test_y))

150000 50000


In [ ]:
# ==============================
import re
from kiwipiepy import Kiwi

kiwi = Kiwi()

def edit_str_21(text):

    def edit(string):
        splited = kiwi.split_into_sents(string)
        res = ['<st>']
        for sent in splited:
            sent = sent.text
            sent = re.sub(r"""[ ]""", "", sent)

            res.extend(sent)
            res.append('<ed>')
        return res
    
    return [edit(string) for string in text]

train_sentences = edit_str_21(train_text_x)
test_sentences = edit_str_21(test_text_x)
for i in train_sentences:
    print(i)

Streaming output truncated to the last 5000 lines.
['<st>', '정', '치', '의', '현', '실', '적', '인', '모', '습', '을', '담', '은', '영', '화', ',', '열', '정', '으', '로', '채', '워', '졌', '던', '스', '티', '븐', '이', '냉', '소', '적', '인', '정', '치', '를', '겪', '은', '후', '변', '하', '는', '모', '습', '을', '소', '름', '끼', '치', '게', '담', '았', '다', '.', "'", '진', '짜', "'", '<ed>', '서', '스', '펜', '스', '스', '릴', '러', '이', '며', '뛰', '어', '난', '배', '우', '들', '의', '연', '기', '를', '보', '는', '재', '미', '가', '있', '었', '다', '.', '<ed>', '남', '자', '는', '여', '자', '관', '계', '를', '조', '심', '해', '야', '하', '는', '법', '.', '<ed>', '정', '치', '의', '이', '면', '.', '<ed>']
['<st>', '인', '사', '이', '드', '아', '읏', '보', '고', '생', '각', '나', '서', '찾', '아', '옴', 'ㅋ', 'ㅋ', 'ㅋ', 'ㅋ', '<ed>', '어', '렸', '을', '때', '봤', '지', '만', '그', '때', '느', '꼈', '던', '감', '동', '은', '아', '직', '도', '생', '생', '함', '<ed>', '로', '봇', '들', '의', '감', '정', '을', '이', '렇', '게', '표', '현', '해', '낼', '수', '있', '다', '니', '.', '.', '<ed>']
['<st>', '9', '0', '년', '대', 'S', 'F', '혁', '

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer(num_words=2500)
tokenizer.fit_on_texts(train_sentences)

train_x = tokenizer.texts_to_sequences(train_sentences)
train_x = pad_sequences(train_x, padding='post', maxlen=140)  # padding='pre': 앞에 패딩 추가
test_x = tokenizer.texts_to_sequences(test_sentences)
test_x = pad_sequences(test_x, padding='post', maxlen=140)  # padding='pre': 앞에 패딩 추가

print(train_x[:5])
print(train_y[:5])

In [ ]:
# ==================================
model = tf.keras.Sequential([
    # 길이 300의 임베딩 벡터로
    tf.keras.layers.Embedding(2500, 300, input_length=140),
    tf.keras.layers.LSTM(units=50),
    tf.keras.layers.Dropout(rate=0.3),
    tf.keras.layers.Dense(2, activation='softmax')
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(train_x, train_y, epochs=50, batch_size=64, validation_split=0.2,
                    callbacks=[tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)])

# ================================
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['loss'], 'b-', label='loss')
plt.plot(history.history['val_loss'], 'r--', label='val_loss')
plt.xlabel('Epoch')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['accuracy'], 'g-', label='accuracy')
plt.plot(history.history['val_accuracy'], 'k--', label='val_accuracy')
plt.xlabel('Epoch')
# plt.ylim(0.7, 1)
plt.legend()

plt.show()

Epoch 1/50
1875/1875 [==============================] - 568s 302ms/step - loss: 0.6935 - accuracy: 0.5015 - f1_score: 0.5015 - val_loss: 0.6931 - val_accuracy: 0.5027 - val_f1_score: 0.5027
Epoch 2/50
1875/1875 [==============================] - 516s 275ms/step - loss: 0.6933 - accuracy: 0.5019 - f1_score: 0.5019 - val_loss: 0.6931 - val_accuracy: 0.5027 - val_f1_score: 0.5027
Epoch 3/50
1875/1875 [==============================] - 504s 269ms/step - loss: 0.6933 - accuracy: 0.5002 - f1_score: 0.5002 - val_loss: 0.6931 - val_accuracy: 0.5027 - val_f1_score: 0.5027
Epoch 4/50
1875/1875 [==============================] - 509s 271ms/step - loss: 0.6932 - accuracy: 0.5009 - f1_score: 0.5009 - val_loss: 0.6931 - val_accuracy: 0.5027 - val_f1_score: 0.5027
Epoch 5/50
1875/1875 [==============================] - 519s 277ms/step - loss: 0.6932 - accuracy: 0.5009 - f1_score: 0.5009 - val_loss: 0.6931 - val_accuracy: 0.5027 - val_f1_score: 0.5027
Epoch 6/50
1875/1875 [============================

In [ ]:
for i in range(20):
    print(test_text_x[i], test_y[i])
    print(model.predict(test_x[i:i + 1], verbose=0)[0])

굳ㅋ [0 1]
[7.7514065e-04 9.9922490e-01]
 [1 0]
[0.4220208 0.5779792]
뭐야이평점들은나쁘진않지만10점짜리는더더욱아니잖아 [1 0]
[0.725873 0.274127]
지루하지는않은데완전막장임돈주고보기에는 [1 0]
[9.9969923e-01 3.0075340e-04]
3만아니었어도별다섯개줬을텐데왜3로나와서제심기를불편하게하죠?? [1 0]
[0.9289815  0.07101853]
음악이주가된최고의음악영화 [0 1]
[0.00422776 0.9957722 ]
진정한쓰레기 [1 0]
[0.9950368  0.00496318]
마치미국애니에서튀어나온듯한창의력없는로봇디자인부터가고개를젖게한다 [1 0]
[0.9942141  0.00578592]
갈수록개판되가는중국영화유치하고내용없음폼잡다끝남말도안되는무기에유치한남무아그립다동사서독같은영화가이건3류아류작이다 [1 0]
[9.9987662e-01 1.2341778e-04]
이별의아픔뒤에찾아오는새로운인연의기쁨모든사람이그렇지는않네 [0 1]
[0.28877094 0.711229  ]
괜찮네요오랜만포켓몬스터잼밌어요 [0 1]
[1.2270584e-04 9.9987733e-01]
한국독립영화의한계그렇게아버지가된다와비교됨 [1 0]
[0.56261176 0.4373882 ]
청춘은아름답다그아름다움은이성을흔들어놓는다찰나의아름다움을잘포착한섬세하고아름다운수채화같은퀴어영화이다 [0 1]
[0.09688645 0.90311354]
눈에보이는반전이었지만영화의흡인력은사라지지않았다 [0 1]
[0.495155   0.50484496]
스토리연출연기비주얼등영화의기본조차안된영화에무슨평을해이런영화찍고도김문옥감독은내가영화경력이몇인데조무래기들이내영화를평론해?같은마인드에빠져있겠지? [1 0]
[0.9878913  0.01210874]
소위ㅈ문가라는평점은뭐냐? [0 1]
[0.8078405  0.19215949]
최고!!!!!!!!!!!!!!!! [0 1]
[1.7030330e-04 9.9982965e-01]
발

In [ ]:
SENT = '''사람들이 쓰레기를 쓰레기통에 버렸다'''

# 가공
SENT = clean_str(SENT)
lst_one = [SENT[:i + 1] for i in range(len(SENT))]
lst = [list(SENT[:i + 1]) for i in range(len(SENT))]
lst = tokenizer.texts_to_sequences(lst)
lst = pad_sequences(lst, padding='post', maxlen=140)  # padding='pre': 앞에 패딩 추가

for i, j in zip(lst, lst_one):
    print(j, model.predict(np.array([i]), verbose=0)[0][1])

사 0.5822158
사람 0.64492875
사람들 0.7922854
사람들이 0.893181
사람들이쓰 0.21998772
사람들이쓰레 0.032976378
사람들이쓰레기 0.013215397
사람들이쓰레기를 0.011502097
사람들이쓰레기를쓰 0.023462975
사람들이쓰레기를쓰레 0.007548652
사람들이쓰레기를쓰레기 0.0058804913
사람들이쓰레기를쓰레기통 0.0056397365
사람들이쓰레기를쓰레기통에 0.0077440785
사람들이쓰레기를쓰레기통에버 0.032295212
사람들이쓰레기를쓰레기통에버렸 0.0071959537
사람들이쓰레기를쓰레기통에버렸다 0.0072855246
